In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 31: Likelihood Ratio Tests

Recall maximum likelihood estimators. These are obtained by maximizing the likelihood function with respect to $\theta$, the parameter of interest. Let's go through an example:

### Example 1: Poisson Distribution

Suppose $\textbf{x} = X_1,X_2,...,X_n$ is an iid sequence of random variables from the Poisson distribution with unknown parameter $\lambda$. Find $\hat{\lambda}_{ML}$, the maximum likelihood estimate of $\lambda$. 

$$
L(\lambda \mid \textbf{x}) = \prod_{i=1}^{n} e^{-\lambda}\frac{\lambda^{x_i}}{x_i!} = \frac{e^{-\lambda n}\lambda^{\Sigma x_i}}{\prod x_i!}
$$  

$$
l(\lambda \mid \textbf{x}) = \log L = -\lambda n + \log{\lambda^{\Sigma x_i}} - \log{\prod x_i!}
$$  

$$
\frac{\text{d}l}{\text{d}\lambda} = -n + \frac{\sum{x_i}}{\lambda} = 0
$$  

$$
\frac{\sum{x_i}}{\lambda} = n
$$  

$$
\Sigma x_i = \lambda n
$$  

$$
\hat{\lambda}_{ML} = \frac{\sum x_i}{n} = \bar{X}
$$

### Likelihood Ratio Tests

Assume you are testing a hypothesis:
$$
H_0: \theta=\theta_0
$$
$$
H_1: \theta\neq \theta_0
$$

The idea behind a likelihood ratio test is to compare the likelihood of the hypothesized value ($L(\theta_0\mid\textbf{x})$) to the maximum likelihood given the data ($L(\hat{\theta}_{ML}\mid\textbf{x})$). If the hypothesized value of $\theta$ were feasible, the likelihood under $\theta_0$ should be close to the max. If the hypothesized value of $\theta$ were not feasible, $L(\theta_0\mid\textbf{x})$ should be much smaller. To make the comparison, we consider the likelihood ratio test statistic, $\Lambda$: 

$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$

Because $\hat{\theta}_{ML}$ is the maximum likelihood estimator, this ratio has a maximum value of 1. Large values of $\Lambda$ (close to 1) indicate that $\theta_0$ is feasible (lack of evidence to reject). Small values of $\Lambda$ (close to 0) indicate $\theta_0$ is not feasible (evidence to reject). 

But how close to 0 is "close"? 

To evaluate this, we will take advantage of a helpful result. It turns out that if the null hypothesis were true, $-2\log \Lambda$ approximately follows the chi-squared distribution with 1 degree of freedom. The proof is outside the scope of this class. 

[We have not yet talked about the chi-squared distribution. To learn more, consult scipy help (`scipy.stats.chi2`). This distribution has one parameter that we care about: degrees of freedom, referenced in scipy as `df`. Bottom line, a random variable that has a chi-squared distribution with `df` degrees of freedom has a domain of $[0,\infty)$ and an expected value of `df`.]

$$
-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right] \approx \textsf{Chisq}(1)
$$

It is a little bit more intuitive to consider $-2\log \Lambda$; large values of this test statistic indicate evidence to reject the null, which is consistent with most other hypothesis tests we've looked at. 

### Example 2: Likelihood ratio test on $\lambda$ from Poisson distribution

(Example taken from Pruim 2011). An instructor believes the the number of students who arrive late for class follows a Poisson process with an average of 1 late arrival per lesson. In 10 consecutive lessons, he found the following number of late arrivals: $\textbf{x} = $ (1,1,0,4,2,1,3,0,0,2). Conduct a likelihood ratio test on the following hypothesis:

$$
H_0: \lambda = 1
$$
$$
H_1: \lambda \neq 1
$$  

$n = 10, \hat{\lambda}_{ML} = \bar{x} = 1.4, \sum x_i = 14$ 
$$
\Lambda = \frac{L(\lambda = 1 \mid \textbf{x})}{L(\lambda = \hat{\lambda}_{ML} \mid \textbf{x})}
$$  
  
$$
-2\log \Lambda = 2\left[l(\lambda = \bar{x}\mid \textbf{x})-l(\lambda = 1\mid \textbf{x})\right]
$$  
  
$$
-2 \log \Lambda = 2\left\{ \left[-(1.4)(10) + 14\log (1.4) - \log(1!1!0!4!2!1!3!0!0!2!)\right] - \left[ -(1)(10) + 14\log(1) - \log(1!1!0!4!2!1!3!0!0!2!)\right]\right\}
$$  
  
$$
-2 \log \Lambda = 2( -14 + 14\log (1.4) + 10 - 14 \log (1) ) = 2(-4 + 14 \log (1.4)) = 1.42
$$

> So the question becomes whether 1.42 is _significantly_ greater than 0. To do this, we compare that value against a $\chi ^2$ distribution with one degree of freedom (because we assumed one parameter in our hypothesis: $\lambda$). If 1.42 is greater than the $\chi^2$-percentile for our chosen significance level (let's say 0.05, i.e. we accept 5% risk of a Type I error (incorrectly rejecting $H_0$)), then we have evidence to reject $H_0$.

In [15]:
stats.chi2.ppf(0.95,1)

3.841458820694124

>The 95th percentile is 3.84. 1.42 is well within that range, therefore we fail to reject (FTR) $H_0$.

#### Alternative Method

In this method, I simply use the sample mean as the test statistic. The observed value was 1.4. In the below, I simulate under the hypothesized $\lambda$, 1, and determine how often the sample mean was further away from 1 as was 1.4. 

In [20]:
ts2=[np.mean(stats.poisson.rvs(1,size=10)) for _ in np.arange(10000)]
np.mean(ts2>=np.mean(x))+np.mean(ts2<=(1-(np.mean(x)-1)))

0.256

> This indicates that, if $\lambda = 1$, our observed value of 1.4 is closer to the "true" (assumed) parameter value of 1 than the test statistic was to 1 in 25.6% of random samples. Therefore, 1.4 is "not that weird" of a value, and we FTR $H_0$.